### Description:

In this project, I built a machine translation system from scratch to translate German sentences into English Using PyTorch and Python from scratch. I created my own encoder-decoder architecture—a type of sequence-to-sequence model. Here’s a simple breakdown:

* Encoder: Reads the German sentence and converts it into a numerical format that captures its meaning.
* Decoder: Uses these numbers and converts it into an English sentence.

I trained and tested my model using the Multi30k dataset, which provides many pairs of German and English sentences. This project highlights my ability to design and build a deep learning model from the ground up without relying too much on pre-built libraries.

This entire Project is built without TorchText library.

#### 1. Importing the Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from datasets import load_dataset
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from nltk.translate.bleu_score import corpus_bleu
import spacy
import random
import math
import numpy as np
import time

c:\USERS\CHALL\ONEDRIVE\DESKTOP\VS CODE DESKTOP\TORCH_ENV\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Checking the availability of GPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


Device Should be set to GPU for faster training.

In [3]:
# To ensure deterministic results

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#### 2. Loading the Data and Preprocessing

In [4]:
# Loading German and English Tokenizers Using spaCy Library

spacy_de = spacy.load("de_core_news_sm")
spacy_en = spacy.load("en_core_web_sm")

In [5]:
# Defining Tokenizer Functions

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [6]:
# Loading the dataset and Mapping it to the Tokenizer Functions

dataset = load_dataset("bentrevett/multi30k")

def tokenize_function(examples):
    return {
        "SRC" : [tokenize_de(example) for example in examples["de"]],
        "TRG" : [tokenize_en(example) for example in examples["en"]],
    }
    
tokenized_dataset = dataset.map(tokenize_function, batched=True)



In [8]:
# Defining Vocabulary with Padding(PAD), Unknown(UNK), Start of Sentence(SOS) and End of Sentence(EOS) Tokens

def build_vocab(tokenized_data, min_freq=2):

    vocab = {"<PAD>": 0, "<UNK>": 1, "<SOS>": 2, "<EOS>": 3}
    idx = 4

    token_counter = Counter(token for tokens in tokenized_data for token in tokens)


    for token, freq in token_counter.items():
        if freq >= min_freq and token not in vocab:
            vocab[token] = idx
            idx += 1

    return vocab


In [10]:
# Building Vocabulary for Source and Target Languages

src_vocab =  build_vocab(tokenized_dataset["train"]["SRC"])
trg_vocab = build_vocab(tokenized_dataset["train"]["TRG"])

In [11]:
# Defining function to add SOS, UNK and EOS tokens.

def numericalize(tokens, vocab):
    return [vocab["<SOS>"]] + [vocab.get(token, vocab["<UNK>"]) for token in tokens] + [vocab["<EOS>"]]

In [12]:
# Defining Function to Numericalize the Tokens

def numericalize_function(examples):
    return {
        "SRC": [numericalize(tokens, src_vocab) for tokens in examples["SRC"]],
        "TRG": [numericalize(tokens, trg_vocab) for tokens in examples["TRG"]]
    }

In [13]:
# Mapping the Numericalize_function to the Tokenized Dataset

numericalized_dataset = tokenized_dataset.map(numericalize_function, batched=True)

In [ ]:
# Defining the collate function to pad the sequences to the same length

def collate_fn(batch):
    src = [torch.tensor(item["SRC"]) for item in batch]
    trg = [torch.tensor(item["TRG"]) for item in batch]
    src_padded = pad_sequence(src, batch_first=True, padding_value=src_vocab["<PAD>"])
    trg_padded = pad_sequence(trg, batch_first=True, padding_value=trg_vocab["<PAD>"])
    return src_padded, trg_padded

In [ ]:
# Data Loaders for Train, Validation and Test Sets

train_loader = DataLoader(numericalized_dataset["train"], batch_size=128, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(numericalized_dataset["validation"], batch_size=128, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(numericalized_dataset["test"], batch_size=128, shuffle=True, collate_fn=collate_fn)

In [ ]:
# Checking the data loader

for (src_batch, trg_batch) in train_loader:
    print("SRC Batch Shape:", src_batch.shape)
    print("TRG Batch Shape:", trg_batch.shape)
    print("Example SRC Batch:", src_batch[0])
    print("Example TRG Batch:", trg_batch[0])
    break

SRC Batch Shape: torch.Size([128, 25])
TRG Batch Shape: torch.Size([128, 26])
Example SRC Batch: tensor([   2,    4,  580,  662,    8,    9,  193,  407,   49,   50,  566,  143,
         204,  841, 3901,   28,    3,    0,    0,    0,    0,    0,    0,    0,
           0])
Example TRG Batch: tensor([   2,   25, 3567, 3899,   52,  568,   49, 1786,   14,    3,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0])


#### 3. Building the Model

In [ ]:
#Encoder Architecture

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout, device):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.device = device 
    
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        #embedded = [batch size, src len, emb dim]
        batch_size = src.shape[0] 

        # Initialize hidden and cell states explicitly
        hidden = torch.zeros(self.n_layers, batch_size, self.hid_dim).to(self.device) 
        cell = torch.zeros(self.n_layers, batch_size, self.hid_dim).to(self.device)
        
        outputs, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # outputs = [batch size, src len, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        
        return hidden, cell
#Here outputs is the hidden states for all time steps in top layer of LSTM.[src_len, batch_size, hid_dim].
#(hidden, cell): Final hidden and cell states from all LSTM layers. Shape of each: [n_layers, batch_size, hid_dim].
#src_len is  sequence length.     

In [ ]:
#Decoder Architecture

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        # input = [batch size]

        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(1)
        # input = [batchsize, 1]
        
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        # seq len and n directions will always be 1 in the decoder, therefore:
        # output = [batch size, 1, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(1))
        # prediction = [batchsize, output_dim]
        
        return prediction, hidden, cell

In [ ]:
# Seq2Seq Model which combines Encoder and Decoder.

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert (
            encoder.hid_dim == decoder.hid_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio=0.5):

        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        input = trg[:, 0]

        for t in range(1, trg_len):

            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[:, t, :] = output

            
            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1

        return outputs

#### 4.Initializing the Hyperparameters and parameters

In [ ]:
# Hyper Parameters for further use

INPUT_DIM = len(src_vocab)
OUTPUT_DIM = len(trg_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
# Initializing the weights of the model

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8014, 256)
    (rnn): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(6191, 256)
    (rnn): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=6191, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
# Checking for number of trainable parameters in the model.

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 14,168,879 trainable parameters


In [ ]:
# Defining the optimizer

optimizer = optim.Adam(model.parameters())

In [ ]:
# Defining the loss function

pad_token = "<PAD>"
PAD_IDX = trg_vocab[pad_token]

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

#### 5. Training the Model

In [ ]:
# Defining the training loop

def train(model, dataloader, optimizer, criterion, clip):
    model.train()  

    epoch_loss = 0

    for batch_idx, (src, trg) in enumerate(dataloader):
        src = src.to(device)
        trg = trg.to(device) 

        optimizer.zero_grad()

        output = model(src, trg)

        output_dim = output.shape[-1]

        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)

        loss = criterion(output, trg)  

        loss.backward()  

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step() 

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)


In [ ]:
# Defining the evaluation loop

def evaluate(model, dataloader, criterion):
    model.eval()  

    epoch_loss = 0

    with torch.no_grad():
        for batch_idx, (src, trg) in enumerate(dataloader):
            src = src.to(device)
            trg = trg.to(device)

            output = model(src, trg, 0)

            output_dim = output.shape[-1]

            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(output, trg)  

            epoch_loss += loss.item()
    
    model.train()

    return epoch_loss / len(dataloader)


In [ ]:
# Defining the function to calculate the elapsed time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

#### 6. Saving the Best Model

In [ ]:
# Training the model and saving the best model

N_EPOCHS = 15
CLIP = 1

best_valid_loss = float("inf")

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_loader, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "Seq model.pt")

    print(f"Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}")
    print(f"\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}")

Epoch: 01 | Time: 0m 31s
	Train Loss: 5.069 | Train PPL: 158.957
	 Val. Loss: 5.083 |  Val. PPL: 161.185
Epoch: 02 | Time: 0m 32s
	Train Loss: 4.483 | Train PPL:  88.495
	 Val. Loss: 4.727 |  Val. PPL: 112.985
Epoch: 03 | Time: 0m 32s
	Train Loss: 4.162 | Train PPL:  64.220
	 Val. Loss: 4.546 |  Val. PPL:  94.232
Epoch: 04 | Time: 0m 32s
	Train Loss: 3.953 | Train PPL:  52.073
	 Val. Loss: 4.433 |  Val. PPL:  84.169
Epoch: 05 | Time: 0m 33s
	Train Loss: 3.790 | Train PPL:  44.272
	 Val. Loss: 4.419 |  Val. PPL:  83.045
Epoch: 06 | Time: 0m 32s
	Train Loss: 3.682 | Train PPL:  39.715
	 Val. Loss: 4.296 |  Val. PPL:  73.390
Epoch: 07 | Time: 0m 31s
	Train Loss: 3.552 | Train PPL:  34.871
	 Val. Loss: 4.239 |  Val. PPL:  69.314
Epoch: 08 | Time: 0m 34s
	Train Loss: 3.421 | Train PPL:  30.587
	 Val. Loss: 4.127 |  Val. PPL:  62.022
Epoch: 09 | Time: 0m 32s
	Train Loss: 3.323 | Train PPL:  27.738
	 Val. Loss: 4.094 |  Val. PPL:  59.961
Epoch: 10 | Time: 0m 33s
	Train Loss: 3.213 | Train PPL

Here we trained the model for 15 epochs and achieved a validation loss of 3.7 and validation perplexity of 44.1. 

#### 7. Evaluating the Model

In [ ]:
# Loading the model for testing

model.load_state_dict(torch.load("Seq model.pt"))

test_loss = evaluate(model, test_loader, criterion)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

C:\Users\chall\AppData\Local\Temp\ipykernel_11796\399348693.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("Seq model.pt"))


| Test Loss: 3.769 | Test PPL:  43.348 |


In [112]:
inv_trg_vocab = {i: w for w, i in trg_vocab.items()}

In [ ]:
# Function to translate the German sentence to English

def translate_sentence(sentence, src_vocab, trg_vocab, model, device, max_len=50):
    
    # Tokenize the German sentence and reverse tokens
    tokens = [tok.text for tok in spacy_de.tokenizer(sentence)][::-1]
    numericalized = (
        [src_vocab["<SOS>"]]
        + [src_vocab.get(token, src_vocab["<UNK>"]) for token in tokens]
        + [src_vocab["<EOS>"]]
    )
    tensor = torch.LongTensor(numericalized).unsqueeze(0).to(device)  # shape: [1, seq_len]

    # Encode the source sentence
    with torch.no_grad():
        hidden, cell = model.encoder(tensor)

    # Initialize the decoder input with <SOS>
    trg_indexes = [trg_vocab["<SOS>"]]

    for _ in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)
        if pred_token == trg_vocab["<EOS>"]:
            break

    # Convert predicted indices to tokens and remove <SOS> and <EOS>
    translated_tokens = [inv_trg_vocab[idx] for idx in trg_indexes]
    return translated_tokens[1:-1]

# Example test sentences (German) and their reference English translations
test_sentences = [
    "Ein kleines Mädchen spielt mit einem Ball im Park.",
    "Ein Mann fährt ein rotes Auto.",
    "Zwei Katzen sitzen auf einem Baum."
]

reference_translations = [
    ["A", "little", "girl", "plays", "with", "a", "ball", "in", "the", "park", "."],
    ["A", "man", "is", "driving", "a", "red", "car", "."],
    ["Two", "cats", "are", "sitting", "in", "a", "tree", "."]
]



In [ ]:
# Translating the test sentences and calculating the BLEU score

predicted_translations = []
for sentence, ref in zip(test_sentences, reference_translations):
    translation = translate_sentence(sentence, src_vocab, trg_vocab, model, device)
    print("Source: ", sentence)
    print("Reference Translation: ", " ".join(ref))
    print("Predicted Translation: ", " ".join(translation))
    print()
    predicted_translations.append(translation)

bleu_score = corpus_bleu([[ref] for ref in reference_translations], predicted_translations)
print("Corpus BLEU score:", bleu_score)

Source:  Ein kleines Mädchen spielt mit einem Ball im Park.
Reference Translation:  A little girl plays with a ball in the park .
Predicted Translation:  A little girl plays playing a a on a beach .

Source:  Ein Mann fährt ein rotes Auto.
Reference Translation:  A man is driving a red car .
Predicted Translation:  A man is riding a a . .

Source:  Zwei Katzen sitzen auf einem Baum.
Reference Translation:  Two cats are sitting in a tree .
Predicted Translation:  Two men sitting on a bench .

Corpus BLEU score: 0.17550802145917324


#### 8.Conclusion

* our German-to-English translation model built with an encoder-decoder architecture achieved a validation loss of 3.7, a perplexity of 44.1, and a BLEU score of 0.17. While these metrics might seem bad at first glance, they are good given the limited training time and resources available for this project. 
* The results demonstrate that even a relatively small and beginner model can capture meaningful patterns in language translation.
* With additional data, further hyperparameter tuning, and potentially more good architectures, there is room for improvement. 

Overall, this project provides a solid foundation and valuable insights for future work in neural machine translation.